In [37]:
import json
import pandas as pd
import folium
import flickrapi as flickr

In [14]:
API_KEY = 'XXXX'
API_SECRET = 'XXXX'
BASE_URL = 'http://farm{farm}.static.flickr.com/{server}/{pid}_{secret}.jpg'

f_api = flickr.FlickrAPI(API_KEY, API_SECRET, format='parsed-json')

def get_geotagged_photos(**args):
    res = f_api.photos.search(**args)
    for photo in res['photos']['photo']:
        url = BASE_URL.format(farm=photo['farm'],
                              server=photo['server'],
                              pid=photo['id'],
                              secret=photo['secret'])
        photo['url'] = url
        geo_data = f_api.photos.geo.getLocation(photo_id=photo['id'])
        photo.update(geo_data['photo']['location'])
        yield photo

#def get_photo_from_url(url):
    

In [15]:
cumbria = get_geotagged_photos(bbox="-3.6, 54.18, -2.72, 54.79",
                               accuracy=16,
                               content_type=1,
                               min_taken_date=1433116800)
data = pd.read_json(json.dumps([record for record in cumbria]))
data.head()

,accuracy,context,country,county,farm,id,isfamily,isfriend,ispublic,latitude,locality,longitude,owner,place_id,region,secret,server,title,url,woeid
0,15,0,"{'woeid': '23424975', '_content': 'United King...","{'woeid': '12602148', '_content': 'Cumbria', '...",4,33318483215,0,0,1,54.468579,"{'woeid': '39235', '_content': 'Wasdale Head',...",-3.254804,53697453@N07,GbqqwR9SW7h9yA,"{'woeid': '24554868', '_content': 'England', '...",d96b71f188,3866,Sunbeam S7 Deluxe,http://farm4.static.flickr.com/3866/3331848321...,39235
1,16,0,"{'woeid': '23424975', '_content': 'United King...","{'woeid': '12602148', '_content': 'Cumbria', '...",4,32475258904,0,0,1,54.580950,"{'woeid': '27872', '_content': 'Loweswater', '...",-3.363342,48779226@N00,OFfXqNZTVbIo5Q,"{'woeid': '24554868', '_content': 'England', '...",e1d797d5a5,3826,Holme Beck,http://farm4.static.flickr.com/3826/3247525890...,27872
2,16,0,"{'woeid': '23424975', '_content': 'United King...","{'woeid': '12602148', '_content': 'Cumbria', '...",1,33317997775,0,0,1,54.576236,"{'woeid': '18258', '_content': 'Dockray', 'pla...",-2.931139,48779226@N00,WoH2qmJQWrjLPg,"{'woeid': '24554868', '_content': 'England', '...",75603dbd42,781,Aira Force,http://farm1.static.flickr.com/781/33317997775...,18258
3,15,0,"{'woeid': '23424975', '_content': 'United King...","{'woeid': '12602148', '_content': 'Cumbria', '...",1,33162511752,0,0,1,54.655029,"{'woeid': '12695900', '_content': 'Allerdale B...",-3.550000,14191740@N08,1MPO0qdQULwUtToCpg,"{'woeid': '24554868', '_content': 'England', '...",8c4f35d9cf,602,"Smew, Siddick Pond, Cumbria, England",http://farm1.static.flickr.com/602/33162511752...,12695900
4,16,0,"{'woeid': '23424975', '_content': 'United King...","{'woeid': '12602148', '_content': 'Cumbria', '...",1,32503576463,0,0,1,54.362351,"{'woeid': '13449', '_content': 'Bowness-on-Win...",-2.923747,14191740@N08,J6upTCxQUb6iDg,"{'woeid': '24554868', '_content': 'England', '...",8a0a65ab8b,684,"Eurasian Jackdaw, Bowness-on-Windermere, Cumbr...",http://farm1.static.flickr.com/684/32503576463...,13449


In [43]:
html = '''
<html>
  <body>
    <h2>{photo_title}</h2>
    <img src="{photo_url}" alt="{photo_title} style="width:400px;">
  </body>
</html>
'''

m = folium.Map(location=[54.5, -3],
               zoom_start=10,
               tiles='Stamen Terrain')
for i, record in data.iterrows():
    iframe = folium.IFrame(html=html.format(photo_title=record.title,
                                            photo_url=record.url),
                           width=500,
                           height=500)
    popup = folium.Popup(html=iframe, max_width=600)
    marker = folium.Marker(location=[record.latitude, record.longitude],
                           popup=popup)
    m.add_child(marker)

m